In [1]:
# I want to download data from Yahoo
# Run a MACD long and short 


In [16]:
import yfinance

In [17]:
import datetime as dt

from nautilus_trader.config import StrategyConfig

from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence

from nautilus_trader.trading.strategy import Strategy

from nautilus_trader.model import BarType
from nautilus_trader.model import Bar
from nautilus_trader.model import Position
from nautilus_trader.model import InstrumentId
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide

class MACDStrategyConfig(StrategyConfig):

    instrument_id: InstrumentId
    bar_type_1day: BarType
    fast_period: int = 4
    slow_period: int = 8
    trading_side = 10_000
    enter_threshold: float = 0.00010

# Create a subclass of Strategy class
class MACDStrategy(Strategy):
    def __init__(self, config: MACDStrategyConfig):
        super().__init__()
        self.bar_type_1day = config.bar_type_1day

        self.macd = MovingAverageConvergenceDivergence(
            fast_period = config.fast_period,
            slow_period = config.slow_period,
            price_type = PriceType.MID
        )
        self.instrument_id = config.instrument_id
        self.trade_size = Quantity.from_int(config.trading_side)
        
        self.position: Position | None = None
        self.enter_threshold = config.enter_threshold
        
        self.count_processed_bars = 0
        
        self.start_time = None
        self.end_time = None

    def on_start(self):
        self.start_time = dt.datetime.now()

        self.subscribe_bars(self.bar_type_1day)

        self.log.info(f"My MACD strategy started at {self.start_time}")

    def on_bar(self, bar: Bar):
    
        self.count_processed_bars += 1

        self.macd.handle_bar(bar)
        if not self.macd.initialized:
            return

        self.check_for_entry()
        self.check_for_exit()

    def check_for_entry(self):
        if self.macd.value >= self.enter_threshold:
            # If already in Long position do nothing
            if self.position and self.position.side == PositionSide.LONG:
                return
            # Else make order
            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.BUY,
                quantity = self.trade_size
            )
            self.submit_order(order)

        elif self.macd.value < -self.enter_threshold:
            # If already in Short poisition do nothing
            if self.position and self.position.side == PositionSide.SHORT:
                return
            # Else make order
            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.SELL,
                quantity = self.trade_size
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If we are in a Short position, exit when MACD value is positive
        if self.macd.value >= 0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        
        # If we are in a Long position, exit when MACD value is negative
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)
        
    
    def on_end(self):
        
        self.end_time = dt.datetime.now()
        self.close_all_positions(instrument_id = self.config.instrument_id)
        self.unsubscribe_bars()
        
        self.log.info(f"My MACD strategy finnished at {self.end_time}")
        self.log.info(f"Total count of 1 day bars: {self.count_processed_bars} ")
    

In [18]:
from decimal import Decimal
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.identifiers import Symbol
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.instruments import Equity
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.objects import Price, Quantity


instrument_id = InstrumentId(
        symbol=Symbol("NVDA"),
        venue=Venue("NASDAQ"),
    )


# Create the NVIDIA equity instrument
NVDA_STOCKS_INSTRUMENT = Equity(
    instrument_id=instrument_id,
    raw_symbol=Symbol("NVDA"),
    currency=USD,
    price_precision=2,  # Prices to 2 decimal places
    price_increment=Price.from_str("0.01"),
    lot_size=Quantity.from_int(1),  # Trading in whole shares
    isin="US67066G1040",  # NVIDIA's ISIN identifier
    ts_event=0,  # Timestamp when the instrument was created/updated
    ts_init=0,   # Timestamp when this object was initialized
)



In [19]:
NVDA_STOCKS_1DAY_BARTYPE = BarType.from_str(
    f"{NVDA_STOCKS_INSTRUMENT.id}-1-DAY-LAST-EXTERNAL"
)



In [20]:
from nautilus_trader.backtest.engine import BacktestEngine

from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.config import ImportableStrategyConfig

from nautilus_trader.model import TraderId
from nautilus_trader.model import Bar

engine_config = BacktestEngineConfig(
    trader_id = TraderId("BACKTEST-NVDA1DAY-001"),
    strategies = [
        ImportableStrategyConfig(
        strategy_path = '__main__:MACDStrategy',
        config_path = '__main__:MACDStrategyConfig',
        config = {
            "instrument_id": instrument_id,
            "bar_type_1day": NVDA_STOCKS_1DAY_BARTYPE,
            "fast_period": 10,
            "slow_period": 20,
            "enter_threshold": 0.00001
        }
        )
    ],
    logging = LoggingConfig(log_level = "DEBUG"),
    
)

engine = BacktestEngine(config = engine_config)

2025-04-27T09:26:15.395932000Z [INFO] BACKTEST-NVDA1DAY-001.BacktestEngine: Building system kernel
2025-04-27T09:26:15.399351000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.database=None
2025-04-27T09:26:15.399360000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.encoding='msgpack'
2025-04-27T09:26:15.399361000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.timestamps_as_iso8601=False
2025-04-27T09:26:15.399419000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.buffer_interval_ms=None
2025-04-27T09:26:15.399431000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.autotrim_mins=None
2025-04-27T09:26:15.399434000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.use_trader_prefix=True
2025-04-27T09:26:15.399436000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.use_trader_id=True
2025-04-27T09:26:15.399437000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.use_instance_id=False
2025-04-27T09:26:15.399438000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: config.streams_prefix='stream

In [21]:
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.objects import Money

engine.add_venue(
        venue=Venue("NASDAQ"),
        oms_type=OmsType.NETTING,  # Order Management System type
        account_type=AccountType.MARGIN,  # Type of trading account
        starting_balances=[Money(1_000_000, USD)],  # Initial account balance
        base_currency=USD,  # Base currency for account
        default_leverage=Decimal(1),  # No leverage used for account
    )


2025-04-27T09:26:15.880543000Z [INFO] BACKTEST-NVDA1DAY-001.SimulatedExchange(NASDAQ): OmsType=NETTING
2025-04-27T09:26:15.882748000Z [INFO] BACKTEST-NVDA1DAY-001.ExecClient-NASDAQ: READY
2025-04-27T09:26:15.883800000Z [INFO] BACKTEST-NVDA1DAY-001.SimulatedExchange(NASDAQ): Registered ExecutionClient-NASDAQ
2025-04-27T09:26:15.883819000Z [INFO] BACKTEST-NVDA1DAY-001.ExecEngine: Registered ExecutionClient-NASDAQ
2025-04-27T09:26:15.884084000Z [INFO] BACKTEST-NVDA1DAY-001.BacktestEngine: Added SimulatedExchange(id=NASDAQ, oms_type=NETTING, account_type=MARGIN)


In [22]:
ydata_df = yfinance.download(tickers=['NVDA'], start='2000-01-01', end='2025-01-01')

# Turn yahoo data into Nautilus Data type

ydata_df.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2000-01-03,0.089431,0.090984,0.084297,0.090268,300912000
2000-01-04,0.087044,0.088118,0.082625,0.087879,300480000
2000-01-05,0.084178,0.085969,0.082984,0.084536,188352000
2000-01-06,0.078685,0.084178,0.075462,0.084178,120480000
2000-01-07,0.079999,0.080835,0.077134,0.078327,71184000


In [23]:
engine.add_instrument(NVDA_STOCKS_INSTRUMENT)

2025-04-27T09:26:16.629374000Z [INFO] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: READY
2025-04-27T09:26:16.629926000Z [INFO] BACKTEST-NVDA1DAY-001.DataEngine: Registered NASDAQ
2025-04-27T09:26:16.630426000Z [DEBUG] BACKTEST-NVDA1DAY-001.Cache: Added instrument NVDA.NASDAQ
2025-04-27T09:26:16.634607000Z [INFO] BACKTEST-NVDA1DAY-001.SimulatedExchange(NASDAQ): Added instrument NVDA.NASDAQ and created matching engine
2025-04-27T09:26:16.634617000Z [INFO] BACKTEST-NVDA1DAY-001.BacktestEngine: Added NVDA.NASDAQ Instrument


In [24]:
from nautilus_trader.persistence.wranglers import BarDataWrangler

wrangler = BarDataWrangler(
    NVDA_STOCKS_1DAY_BARTYPE,
    NVDA_STOCKS_INSTRUMENT
)

nvda_1day_bars_list: list[Bar] = wrangler.process(ydata_df)



2025-04-27T09:26:18.221489000Z [INFO] BACKTEST-NVDA1DAY-001.BacktestEngine: Added 6_289 NVDA.NASDAQ-1-DAY-LAST-EXTERNAL Bar elements


In [25]:
#strategy = MACDStrategy(config=MACDStrategyConfig)
#engine.add_strategy(strategy)

# Execute the backtest
engine.run()



2025-04-27T09:26:19.216905000Z [DEBUG] BACKTEST-NVDA1DAY-001.Cache: Added Account(id=NASDAQ-001)
2025-04-27T09:26:19.217158000Z [DEBUG] BACKTEST-NVDA1DAY-001.Cache: Indexed AccountId('NASDAQ-001')
2025-04-27T09:26:19.217457000Z [INFO] BACKTEST-NVDA1DAY-001.Portfolio: Updated AccountState(account_id=NASDAQ-001, account_type=MARGIN, base_currency=USD, is_reported=True, balances=[AccountBalance(total=1_000_000.00 USD, locked=0.00 USD, free=1_000_000.00 USD)], margins=[], event_id=922e72ac-7a77-4b5b-b073-f0a555eab75c)
2000-01-03T00:00:00.000000000Z [INFO] BACKTEST-NVDA1DAY-001.BacktestEngine: STARTING
2000-01-03T00:00:00.000000000Z [DEBUG] BACKTEST-NVDA1DAY-001.DataEngine: STARTING
2000-01-03T00:00:00.000000000Z [DEBUG] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: STARTING
2000-01-03T00:00:00.000000000Z [INFO] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: Connecting...
2000-01-03T00:00:00.000000000Z [INFO] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: Connected
2000-01-03T00:00:00.000000000Z [INFO] BACK

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
# Clean up resources
engine.dispose()

2025-04-27T09:26:44.163175000Z [DEBUG] BACKTEST-NVDA1DAY-001.DataEngine: DISPOSING
2025-04-27T09:26:44.163364000Z [DEBUG] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: DISPOSING
2025-04-27T09:26:44.163825000Z [INFO] BACKTEST-NVDA1DAY-001.DataClient-NASDAQ: DISPOSED
2025-04-27T09:26:44.163863000Z [INFO] BACKTEST-NVDA1DAY-001.DataEngine: DISPOSED
2025-04-27T09:26:44.163935000Z [DEBUG] BACKTEST-NVDA1DAY-001.RiskEngine: DISPOSING
2025-04-27T09:26:44.163971000Z [INFO] BACKTEST-NVDA1DAY-001.RiskEngine: DISPOSED
2025-04-27T09:26:44.164035000Z [DEBUG] BACKTEST-NVDA1DAY-001.ExecEngine: DISPOSING
2025-04-27T09:26:44.164142000Z [DEBUG] BACKTEST-NVDA1DAY-001.ExecClient-NASDAQ: DISPOSING
2025-04-27T09:26:44.164160000Z [INFO] BACKTEST-NVDA1DAY-001.ExecClient-NASDAQ: DISPOSED
2025-04-27T09:26:44.164189000Z [INFO] BACKTEST-NVDA1DAY-001.ExecEngine: DISPOSED
2025-04-27T09:26:44.164725000Z [DEBUG] BACKTEST-NVDA1DAY-001.MessageBus: Closing message bus
4.164732000Z [INFO] BACKTEST-NVDA1DAY-001.MessageBus: Close

In [28]:
engine.trader.generate_orders_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20000131-000000-001-000-1,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-001,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-021,MARKET,SELL,10000,...,None,None,None,None,None,None,None,2921d942-17b4-4c86-a2a3-b5987952087e,949276800000000000,949276800000000000
O-20000201-000000-001-000-2,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-002,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-023,MARKET,SELL,10000,...,None,None,None,None,None,None,None,8e8f3c29-ff07-40f3-afd7-e02b52fcfeee,949363200000000000,949363200000000000
O-20000202-000000-001-000-3,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-003,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-025,MARKET,SELL,10000,...,None,None,None,None,None,None,None,83b05d72-c3f8-48b5-a268-b146105bbc39,949449600000000000,949449600000000000
O-20000203-000000-001-000-4,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-004,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-027,MARKET,SELL,10000,...,None,None,None,None,None,None,None,9d9c39fc-74aa-4f14-b8de-8bd473d1a992,949536000000000000,949536000000000000
O-20000204-000000-001-000-5,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-005,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-029,MARKET,SELL,10000,...,None,None,None,None,None,None,None,1416821c-6650-47cd-9456-dfbb754f77c0,949622400000000000,949622400000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241224-000000-001-000-6263,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-6263,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-12548,MARKET,SELL,10000,...,None,None,None,None,None,None,None,ee9791d0-09f5-46f7-a5d5-7db2036d9d3e,1734998400000000000,1734998400000000000
O-20241226-000000-001-000-6264,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-6264,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-12550,MARKET,SELL,10000,...,None,None,None,None,None,None,None,0b34331f-8585-4851-a4b1-4abf757bc5cc,1735171200000000000,1735171200000000000
O-20241227-000000-001-000-6265,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-6265,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-12552,MARKET,SELL,10000,...,None,None,None,None,None,None,None,724b048a-575a-491c-9253-db56077b8879,1735257600000000000,1735257600000000000


In [27]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
NVDA.NASDAQ-MACDStrategy-000-73140ce0-3f8b-42e3-b01f-f9c61624f78a,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20000131-000000-001-000-1,O-20000224-000000-001-000-18,SELL,FLAT,0,90000,949276800000000000,2000-01-31 00:00:00+00:00,951350400000000000,2000-02-24 00:00:00+00:00,2.073600e+15,0.075556,0.112222,[0.00 USD],-0.48529,-3299.96 USD
NVDA.NASDAQ-MACDStrategy-000-95b682c0-01e6-4f2b-9477-685341d20ea7,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20000225-000000-001-000-19,O-20021002-000000-001-000-670,BUY,FLAT,0,1590000,951436800000000000,2000-02-25 00:00:00+00:00,1033516800000000000,2002-10-02 00:00:00+00:00,8.208000e+16,0.352514,0.227454,[0.00 USD],-0.35477,-235300.15 USD
NVDA.NASDAQ-MACDStrategy-000-ded8650e-54c1-48bf-976a-6d235ffc5f0c,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20021003-000000-001-000-671,O-20021111-000000-001-000-698,SELL,FLAT,0,140000,1033603200000000000,2002-10-03 00:00:00+00:00,1036972800000000000,2002-11-11 00:00:00+00:00,3.369600e+15,0.066429,0.095000,[0.00 USD],-0.43011,-3999.94 USD
NVDA.NASDAQ-MACDStrategy-000-cf857222-e8b2-425b-a1f4-13bb2e63b42d,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20021112-000000-001-000-699,O-20030115-000000-001-000-742,BUY,FLAT,0,220000,1037059200000000000,2002-11-12 00:00:00+00:00,1042588800000000000,2003-01-15 00:00:00+00:00,5.529600e+15,0.112273,0.096818,[0.00 USD],-0.13765,-3400.06 USD
NVDA.NASDAQ-MACDStrategy-000-0fb0e733-6645-4875-84e0-d16b7c3a52f5,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20030116-000000-001-000-743,O-20030318-000000-001-000-784,SELL,FLAT,0,210000,1042675200000000000,2003-01-16 00:00:00+00:00,1047945600000000000,2003-03-18 00:00:00+00:00,5.270400e+15,0.080000,0.097143,[0.00 USD],-0.21429,-3600.00 USD
NVDA.NASDAQ-MACDStrategy-000-836ec644-21bc-44a0-ba21-65290824da07,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20030319-000000-001-000-785,O-20040812-000000-001-000-1138,BUY,FLAT,0,790000,1048032000000000000,2003-03-19 00:00:00+00:00,1092268800000000000,2004-08-12 00:00:00+00:00,4.423680e+16,0.172529,0.146271,[0.00 USD],-0.15219,-26399.85 USD
NVDA.NASDAQ-MACDStrategy-000-9772333f-88f7-4a42-b65a-977c3831b25d,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20040813-000000-001-000-1139,O-20040930-000000-001-000-1172,SELL,FLAT,0,170000,1092355200000000000,2004-08-13 00:00:00+00:00,1096502400000000000,2004-09-30 00:00:00+00:00,4.147200e+15,0.092941,0.108235,[0.00 USD],-0.16456,-2600.03 USD
NVDA.NASDAQ-MACDStrategy-000-f810a09b-7e89-46b8-a8f4-09dd915c07fd,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20041001-000000-001-000-1173,O-20041101-000000-001-000-1194,BUY,FLAT,0,110000,1096588800000000000,2004-10-01 00:00:00+00:00,1099267200000000000,2004-11-01 00:00:00+00:00,2.678400e+15,0.114545,0.107273,[0.00 USD],-0.06349,-799.95 USD
NVDA.NASDAQ-MACDStrategy-000-1113b44f-47d5-4182-a426-b8411ffde0eb,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20041102-000000-001-000-1195,O-20041103-000000-001-000-1196,SELL,FLAT,0,10000,1099353600000000000,2004-11-02 00:00:00+00:00,1099440000000000000,2004-11-03 00:00:00+00:00,8.640000e+13,0.110000,0.120000,[0.00 USD],-0.09091,-100.00 USD
